In [5]:
# ! pip install opencv-python
# ! pip install tensorflow==1.13.1

In [6]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# from tensorflow.python.framework import graph_util
# from tensorflow.python.platform import gfile

# tf.logging.set_verbosity(tf.logging.ERROR)

# mnist = input_data.read_data_sets('./MNIST_data/', one_hot=True)

# #
# # hyper parameters
# #
# learning_rate = 0.001
# training_epochs = 20
# batch_size = 100

# #
# # Model configuration
# #
# X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='data')
# Y = tf.placeholder(tf.float32, [None, 10])

# conv1 = tf.layers.conv2d(X, 10, [3, 3], padding='same', activation=tf.nn.relu)
# pool1 = tf.layers.max_pooling2d(conv1, [2, 2], strides=2, padding='same')

# conv2 = tf.layers.conv2d(pool1, 20, [3, 3], padding='same', activation=tf.nn.relu)
# pool2 = tf.layers.max_pooling2d(conv2, [2, 2], strides=2, padding='same')

# fc1 = tf.contrib.layers.flatten(pool2)
# fc2 = tf.layers.dense(fc1, 200, activation=tf.nn.relu)
# logits = tf.layers.dense(fc2, 10, activation=None)
# output = tf.nn.softmax(logits, name='prob')

# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# #
# # Training
# #
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# total_batch = int(mnist.train.num_examples / batch_size)

# print('Start learning!')
# for epoch in range(training_epochs):
#     total_cost = 0

#     for i in range(total_batch):
#         batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#         batch_xs = batch_xs.reshape(-1, 28, 28, 1)
#         _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
#         total_cost += cost_val

#     print('Epoch: {0}, Avg. Cost = {1:.4f}'.format(epoch + 1, total_cost/total_batch))

# print('Learning finished!')

# # Test the results
# is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
# acc = tf.reduce_mean(tf.cast(is_correct, tf.float32))
# accuracy = sess.run(acc, feed_dict={
#                     X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels})
# print('Test Accuracy:', accuracy)

# # Freeze variables and save pb file
# output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['prob'])
# with gfile.FastGFile('./mnist_cnn.pb', 'wb') as f:
#     f.write(output_graph_def.SerializeToString())

# print('mnist_cnn.pb file is created successfully!!')


## Deep learning을 이용한 필기체 인식

In [7]:
# cv2.dnn.readNet(model, config=None, framework=None) -> retval
    # model:훈련된 가중치 파일
    # config: 네트워크 구성을 저장하고 있는 텍스트 파일 이름
    # framework: None
    # retval: cv2.dnn_Net 클래스 객체
    
# cv2.dnn.blobFromImage(image, scalefactor=None, size=None, mean=None,
#                         swapRB=None, crop=None, ddepth=None)
    # image: 입력영상
    # scalefactor: 입력영상 픽셀에 곱할 값, 기본값은 1
    # size: 츨력영상의 크기, 기본값은 (0,0)
    # mean: 입력영상 각 채널에서 뺄 평균값
    # swapRB: R,B 채널의 순서, True or False
    # ddepth: 출력영상의 깊이 CV_32F
    # retval:blob객체, numpy.ndarray, shape=(N,C,H,W), dtype=numpy.float32

# net.setInput(blob, name=None, scalefactor=None, mean=None)-> None
    # blob: blob객체
    # name
    # scalefactor
    # mean

# net.forward(outputName=None) -> retval
    # outputName: 출력레이어 이름
    # retval: dnn 결과

In [9]:
import sys
import numpy as np
import cv2


oldx, oldy = -1, -1


def on_mouse(event, x, y, flags, _):
    global oldx, oldy

    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)


net = cv2.dnn.readNet('mnist_cnn.pb')

if net.empty():
    print('Network load failed!')
    sys.exit()

img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)

while True:
    c = cv2.waitKey()

    if c == 27:
        break
    elif c == ord(' '):
        blob = cv2.dnn.blobFromImage(img, 1/255., (28, 28))
        net.setInput(blob)
        prob = net.forward()

        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        digit = maxLoc[0] # 판단된

        print(f'{digit} ({maxVal * 100:4.2f}%)')

        img.fill(0)
        cv2.imshow('img', img)

cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'numpy'

## GoogleNet 영상인식 모델 실행

In [ ]:
# https://modelzoo.co/model/googlenet-v2 모델파일 다운로드

# Caffe Model Zoo : github.com/BVLC/caffe
## 모델 파일 : dl.caffe.berkeleyvision.org/bvlc_googlenet.caffemodel
## 설정 파일 : github.com/BVLC/caffe/blob/master/models/bvlc_googlenet/deploy.prototxt


# ONNX Model Zoo : github.com/onnx/models

# 클래스 이름 파일 : github.com/opencv/opencv/blob/4.1.0/samples/data/dnn/classification_classes_ILSVRC2012.txt

In [ ]:
import sys
import numpy as np
import cv2


# 입력 영상 불러오기

# filename = 'apple1.png'
filename = './googlenet/pineapple.jpg'

# if len(sys.argv) > 1: 
#     filename = sys.argv[1]

img = cv2.imread(filename)

if img is None:
    print('Image load failed!')
    sys.exit()

# 네트워크 불러오기

# Caffe
model = 'googlenet/bvlc_googlenet.caffemodel'
config = 'googlenet/deploy.prototxt.txt'

# ONNX
# model = 'googlenet/googlenet-9.onnx'
# config = ''

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Network load failed!')
    sys.exit()

# 클래스 이름 불러오기

classNames = []
with open('googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

# 추론

blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123))
net.setInput(blob)
prob = net.forward()

# 추론 결과 확인 & 화면 출력

out = prob.flatten()
classId = np.argmax(out)
confidence = out[classId]

text = f'{classNames[classId]} ({confidence * 100:4.2f}%)'
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()


## OpenCV DNN 얼굴검출

In [ ]:
# https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector
# deploy.prototxt.txt, download-weights.py.txt, opencv_face_detector.pbtxt.text 다운로드

# Caffe    https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel

# Tensorflow  https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180220_uint8/opencv_face_detector_uint8.pb



In [ ]:
import sys
import numpy as np
import cv2


# model = 'opencv_face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel'
# config = 'opencv_face_detector/deploy.prototxt'
model = 'opencv_face_detector/opencv_face_detector_uint8.pb'
config = 'opencv_face_detector/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Net open failed!')
    sys.exit()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()# out.shape=(1,1, 200, 7)

    detect = out[0, 0, :, :] ##0, 0, 사용안함
    (h, w) = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        if confidence < 0.5:
            break

        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))

        label = f'Face: {confidence:4.2f}'
        cv2.putText(frame, label, (x1, y1 - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()


## YOLOv3를 이용한 객체 검출

In [ ]:
# https://pjreddie.com/darknet/yolo/

In [ ]:
import sys
import numpy as np
import cv2


# 모델 & 설정 파일
model = 'yolo_v3/yolov3.weights'
config = 'yolo_v3/yolov3.cfg'
class_labels = 'yolo_v3/coco.names'
confThreshold = 0.5
nmsThreshold = 0.4

# 테스트 이미지 파일
img_files = ['yolo_v3/dog.jpg', 'yolo_v3/person.jpg', 
             'yolo_v3/sheep.jpg', 'yolo_v3/kite.jpg']

# 네트워크 생성
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Net open failed!')
    sys.exit()

# 클래스 이름 불러오기

classes = []
with open(class_labels, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

colors = np.random.uniform(0, 255, size=(len(classes), 3))

# 출력 레이어 이름 받아오기

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# output_layers = ['yolo_82', 'yolo_94', 'yolo_106']

# 실행

for f in img_files:
    img = cv2.imread(f)

    if img is None:
        continue

    # 블롭 생성 & 추론
    blob = cv2.dnn.blobFromImage(img, 1/255., (320, 320), swapRB=True)
    net.setInput(blob)
    outs = net.forward(output_layers) #

    # outs는 3개의 ndarray 리스트.
    # outs[0].shape=(507, 85), 13*13*3=507
    # outs[1].shape=(2028, 85), 26*26*3=2028
    # outs[2].shape=(8112, 85), 52*52*3=8112

    h, w = img.shape[:2]

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            # detection: 4(bounding box) + 1(objectness_score) + 80(class confidence)
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confThreshold:
                # 바운딩 박스 중심 좌표 & 박스 크기
                cx = int(detection[0] * w)
                cy = int(detection[1] * h)
                bw = int(detection[2] * w)
                bh = int(detection[3] * h)

                # 바운딩 박스 좌상단 좌표
                sx = int(cx - bw / 2)
                sy = int(cy - bh / 2)

                boxes.append([sx, sy, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))

    # 비최대 억제, Non Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

    for i in indices:
        i = i[0]
        sx, sy, bw, bh = boxes[i]
        label = f'{classes[class_ids[i]]}: {confidences[i]:.2}'
        color = colors[class_ids[i]]
        cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
        cv2.putText(img, label, (sx, sy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 255), 1, cv2.LINE_AA)

    cv2.imshow('img', img)
    cv2.waitKey()

cv2.destroyAllWindows()
